<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Logo de Skills Network">
    </a>
</p>


# **Predicción de Aterrizaje de la Primera Etapa del Falcon 9 de Space X**


## Web scraping de los registros de lanzamientos de Falcon 9 y Falcon Heavy desde Wikipedia


Tiempo estimado necesario: **40** minutos


En este laboratorio, realizarás web scraping para recolectar registros históricos de lanzamientos de Falcon 9 desde una página de Wikipedia titulada `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


La primera etapa del Falcon 9 aterrizará con éxito


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Varios ejemplos de un aterrizaje fallido se muestran aquí:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


Más específicamente, los registros de lanzamiento se almacenan en una tabla HTML que se muestra a continuación:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objetivos
Hacer web scraping de los registros de lanzamiento de Falcon 9 con `BeautifulSoup`: 
- Extraer una tabla HTML de registros de lanzamiento de Falcon 9 de Wikipedia
- Analizar la tabla y convertirla en un DataFrame de Pandas


Primero, importemos los paquetes necesarios para este laboratorio


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

y te proporcionaremos algunas funciones de ayuda para procesar la tabla HTML extraída mediante web scraping


In [3]:
def date_time(table_cells):
    """
    Esta función devuelve la fecha y la hora de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla extrae una fila adicional
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    Esta función devuelve la versión del propulsor desde la celda de la tabla HTML 
    Entrada: el elemento de una celda de datos de la tabla extrae una fila adicional
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    Esta función devuelve el estado del aterrizaje desde la celda de la tabla HTML 
    Entrada: el elemento de una celda de datos de la tabla extrae una fila adicional
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    Esta función extrae el texto de un encabezado de tabla
    Entrada: el elemento th de una fila de la tabla
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filtrar los nombres vacíos y los que son dígitos
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


Para mantener la coherencia de las tareas del laboratorio, se te pedirá que extraigas los datos de una instantánea de la página de Wikipedia `List of Falcon 9 and Falcon Heavy launches` actualizada el
`9 de junio de 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

A continuación, solicita la página HTML de la URL anterior y obtén un objeto `response`


### TAREA 1: Solicitar la página de la Wiki de lanzamientos de Falcon9 desde su URL


Primero, realicemos una solicitud HTTP con el método GET para solicitar la página HTML de lanzamientos de Falcon9, como una respuesta HTTP.


In [5]:
# usa el método requests.get() con la static_url y los headers proporcionados
# asigna la respuesta a un objeto
response = requests.get(static_url, headers=headers)

Crea un objeto `BeautifulSoup` a partir de la respuesta HTML (`response`)


In [6]:
# Usa BeautifulSoup() para crear un objeto BeautifulSoup a partir del contenido de texto de una respuesta
soup = BeautifulSoup(response.text, 'html.parser')

Imprime el título de la página para verificar si el objeto `BeautifulSoup` se creó correctamente 


In [7]:
# Usa el atributo soup.title
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TAREA 2: Extraer todos los nombres de columnas/variables del encabezado de la tabla HTML


A continuación, queremos recolectar todos los nombres de columnas relevantes del encabezado de la tabla HTML


Intentemos primero encontrar todas las tablas en la página de la wiki. Si necesitas refrescar tu memoria sobre `BeautifulSoup`, por favor revisa el enlace de referencia externa hacia el final de este laboratorio


In [8]:
# Usa la función find_all en el objeto BeautifulSoup, con el tipo de elemento `table`
# Asigna el resultado a una lista llamada `html_tables`
html_tables = soup.find_all('table')

A partir de la tercera tabla se encuentra nuestra tabla objetivo que contiene los registros de lanzamiento reales.


In [9]:
# Imprimamos la tercera tabla y verifiquemos su contenido
first_launch_table = html_tables[2]
print(first_launch_table.prettify())

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
  

Deberías poder ver los nombres de las columnas incrustados en los elementos del encabezado de la tabla `<th>` de la siguiente manera:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


A continuación, solo necesitamos iterar a través de los elementos `<th>` y aplicar la función `extract_column_from_header()` proporcionada para extraer el nombre de cada columna uno por uno


In [10]:
column_names = []

# Aplica la función find_all() con el elemento `th` en first_launch_table
th_elements = first_launch_table.find_all('th')
# Itera cada elemento th y aplica la función extract_column_from_header() proporcionada para obtener un nombre de columna
for th in th_elements:
    name = extract_column_from_header(th)
    # Agrega el nombre de columna no vacío (`if name is not None and len(name) > 0`) a una lista llamada column_names
    if name is not None and len(name) > 0:
        column_names.append(name)

Revisa los nombres de las columnas extraídas


In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TAREA 3: Crear un DataFrame analizando las tablas HTML de lanzamiento


Crearemos un diccionario vacío con claves de los nombres de las columnas extraídas en la tarea anterior. Más tarde, este diccionario se convertirá en un DataFrame de Pandas


In [12]:
launch_dict= dict.fromkeys(column_names)

# Eliminar una columna irrelevante
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

# Inicialicemos el launch_dict con cada valor como una lista vacía
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Se agregaron algunas columnas nuevas
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

A continuación, solo necesitamos llenar el `launch_dict` con los registros de lanzamiento extraídos de las filas de la tabla.


Por lo general, es probable que las tablas HTML en las páginas de la Wiki contengan anotaciones inesperadas y otros tipos de ruido, como enlaces de referencia `B0004.1[8]`, valores faltantes `N/A [e]`, formato inconsistente, etc.


Para simplificar el proceso de análisis, hemos proporcionado un fragmento de código incompleto a continuación para ayudarte a llenar el `launch_dict`. Por favor, completa el siguiente fragmento de código con los TODOs o puedes optar por escribir tu propia lógica para analizar todas las tablas de lanzamiento:


In [13]:
extracted_row = 0
#Extraer cada tabla 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # obtener la fila de la tabla 
    for rows in table.find_all("tr"):
        # verificar si el primer encabezado de la tabla es un número correspondiente a un número de lanzamiento 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        # obtener el elemento de la tabla 
        row=rows.find_all('td')
        # si es un número, guardar las celdas en un diccionario 
        if flag:
            extracted_row += 1
            # Valor del número de vuelo
            launch_dict['Flight No.'].append(flight_number)
            
            datatimelist=date_time(row[0])
            
            # Valor de la fecha
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Valor de la hora
            time = datatimelist[1]
            launch_dict['Time'].append(time)
              
            # Versión del propulsor
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string if row[1].a else ''
            launch_dict['Version Booster'].append(bv)
            
            # Sitio de lanzamiento
            launch_site = row[2].a.string if row[2].a else ''
            launch_dict['Launch site'].append(launch_site)
            
            # Carga útil
            payload = row[3].a.string if row[3].a else ''
            launch_dict['Payload'].append(payload)
            
            # Masa de la carga útil
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Órbita
            orbit = row[5].a.string if row[5].a else ''
            launch_dict['Orbit'].append(orbit)
            
            # Cliente
            customer = row[6].a.string if row[6].a else ''.join(row[6].stripped_strings)
            launch_dict['Customer'].append(customer)
            
            # Resultado del lanzamiento
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Aterrizaje del propulsor
            booster_landing = landing_status(row[8]).strip()
            launch_dict['Booster landing'].append(booster_landing)


Después de haber llenado los valores de los registros de lanzamiento analizados en `launch_dict`, puedes crear un DataFrame a partir de él.


In [14]:
df= pd.DataFrame(launch_dict)

Ahora podemos exportarlo a un <b>CSV</b> para la siguiente sección. Los laboratorios siguientes utilizarán este conjunto de datos para que cada laboratorio sea independiente.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>

In [15]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Autores


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


Copyright © 2021 IBM Corporation. Todos los derechos reservados.
